In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import io
def get_img_from_fig(fig, dpi=90):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    return img

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
cap=cv2.VideoCapture(r'C:\Users\17\Downloads\video5\1.mp4')
out = cv2.VideoWriter('outpy3.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 5, (2520,1070))
corona_count=[]
img_counter=0
while True:
    ret, img = cap.read()
    if ret:
        cv2.imwrite(r'C:\Users\17\Downloads\video5\images\frame_'+str(img_counter).zfill(6)+'.png',img[50:-50,10:-10,:])
        img=img[50:-50,10:-10,:]
        img_counter+=1
        mask = cv2.inRange(img, (0,0,190), (75,75,255))
        corona_count.append(mask[mask>0].shape[0])
        xy=np.where(mask>0)
        points=np.swapaxes(np.array(xy),0,1)
        if points.shape[0]>10:
            clustering = DBSCAN(eps=30, min_samples=30).fit(points)
            lab=clustering.labels_
            img4=np.zeros(img.shape)
            colors=[(100,100,100),(255,0,0),(0,255,0),(0,0,255),(150,150,150)]
            for idx,label in enumerate(clustering.labels_):
                img4[points[idx][0],points[idx][1]]=colors[label]
        else:
            img4=np.zeros(img.shape)
        res_img=np.zeros((img.shape[0]+450,img.shape[1]*2,img.shape[2]))
        res_img[:620,:img.shape[1],:]=img.copy()
        res_img[:620,img.shape[1]:,:]=img4.copy()
        
        fig=plt.figure(figsize=(15,5))
        fig.tight_layout(pad=0)
        plot   = fig.add_subplot ( 111 )
        plot.plot(np.stack(corona_count))
        
        #plt.show(block=True)
        #plt.savefig('1.jpg')
        d=get_img_from_fig(fig)
        res_img[620:,:d.shape[1],:]=d
        res_img=res_img.astype(np.uint8)
        out.write(res_img)
        if img_counter>9:
            break
    else:
        break
cap.release()
out.release()       